# From peptide and spectrum to user and item

I'm trying to illustrate the connection between a project that I worked on for my PhD (my research area was computational mass spectrometry based proteomics, [[1]](https://cs.uwaterloo.ca/~binma/cs882/4.pdf), [[2]](https://en.wikipedia.org/wiki/Protein_mass_spectrometry)) and the collaborative filtering algorithm used in recommender systems (e.g. Netflix, YouTube).

## 1. Peptide and spectrum

In a typical workflow of *computational proteomics*, the most important step is finding the "best match" in a fixed list of potential candidate [*peptides*](https://en.wikipedia.org/wiki/Peptide) against a fragmentation *spectrum* (think of the *spectrum* as the query of a google search, and *peptides* as the list of returned links). The *spectrum* is obtained by running analytical chemistry experiment on biological sample which contains the *best-match peptide*, and it can be conceptually represented as a list of tuples containing two quantities (the *mass-to-charge-ratio* and peak *intensity*, see the figure below on the right). This *peptide-to-spectrum* match can be achieved simply because the *spectrum* contains information that is specific to the *best-match peptide* (e.g. think of fingers and fingerprints).

To carry out the *peptide-to-spectrum* match, we often need to come up with an *approximated spectrum* for each *peptide* in the list of candidates. Theoretically this can be done because there is a deterministic generative process from *peptide* to *spectrum*. However, computing the *approximated spectrum* is difficult because in practice it's hard to directly model the relation from *peptide* and *spectrum*.

### 1.1 Connection to recommder system
I find surprisingly interesting that the "generating-spectrum-from-peptide" step is very similar to the situation of *recommending items to users*. You can roughly think of *peptide* as *user*, and *spectrum* as *items* liked by the *user*. The "generating-spectrum-from-peptide" step is conceptually similar to finding the recommended items for a given *user*. It's also interesting to note that it's also generally hard to directly model the user-to-item relationship (e.g. for the content-based filtering).

<img src="1.png">

## 2. Neighbor-based approach

The solution to the problem is, just like the [memory-based collaborative filtering](https://en.wikipedia.org/wiki/Collaborative_filtering#Memory-based), given a peptide $p_{q}$, find the closest neighboring peptides (like *users*) $p_{1}, p_{2}, \cdots, p_{k}$, and average their *spectra* (list of liked *items*) as the recommended "spectrum" for $p_{q}$.


<img src="2.png">

## 3. Distance metric over peptides

The key to the success of the neighbor-based approach is to find an appropriate distance metric over all peptides, so you can weight the contribution of neighboring peptides in the generated *spectrum* for *peptide* $p_{q}$. By *appropriate*, I mean the distance metric on *peptides* $p_{i}$ and $p_{j}$ should be low if the corresponding *spectra* $s_{i}$ and $s_{j}$ are similar (for example, in terms of cosine or Pearson correlation, because $s_{i}$ and $s_{j}$ can be represented as vectors of intensities).

<img src="3.png">

In a memory-based collaborative filtering algorithm (user-centered), computing the distance metric over users is very simple: each user can be characterized by a vector containing their ratings over the times, so distance between users can be computed as the Euclidean or cosine distance.

However, in the case of *peptide* and *spectrum*, there is no *rating vectors* for *peptides*, so we need to find the workaround for computing the distance metric over *peptides*.

## 4. Learning distance metric

Remember that the distance metric (or distance function) operates on *pairs* of *peptides* $(p_{i}, p_{j})$, and output a low value if the corresponding *spectra* $(s_{i}, s_{j})$ are similar. In light of this observation, we can train a SVM model over instances of *pairs* of *peptides*, where the positive class consists of *similar peptides*, and negative class *dissimilar peptides*.

Conventionally, the output from a SVM model is used to assign predicted label on a test instance (positive if greater than zero, and negative if less than zero). But the output is also an indicator of the *distance* of the test instance with respect to the positve or negatice class.

So given a test instance containing peptide pairs $(p_{u}, p_{v})$, the greater the output from SVM model $\sigma(p_{u}, p_{v})$, the lower the distance metric would be.

<img src="4.png">